### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup

import pandas as pd
pd.set_option('mode.chained_assignment', None)

# File to Load (Remember to Change These)
school_data_to_load = "../Instructions/PyCitySchools/Resources/schools_complete.csv"
student_data_to_load = "../Instructions/PyCitySchools/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [34]:
total_schools= len(school_data)
total_student= len(student_data)
total_budget= sum(school_data["budget"])
average_math= student_data["math_score"].mean()

# Percentage of students with a passing math score (70 or greater)
total_passing_math= student_data.loc[student_data["math_score"] >= 70, :]
percent_passing_math= len(total_passing_math)/total_student*100
percent_pass_math_round= round(percent_passing_math)

# Percentage of students with a passing reading score (70 or greater)
total_passing_reading= student_data.loc[student_data["reading_score"] >=70, :]
percent_passing_reading= len(total_passing_reading)/total_student*100
percent_pass_reading_round= round(percent_passing_reading)

#percentage of students passing both
total_passing_both= student_data.loc[(student_data["reading_score"] >=70) & (student_data["math_score"] >= 70), :]
percent_passing_both= len(total_passing_both)/total_student*100
percent_pass_both_round= round(percent_passing_both)


summary_dict= {"Total Schools":[total_schools], "Total Students":[total_student], "Total Budget":[total_budget],
               "Avg Math Score":[average_math], "% Passing Math":[percent_pass_math_round], 
               "% Passing Reading":[percent_pass_reading_round], "% Passing Both":[percent_pass_both_round]}
summary_df= pd.DataFrame(data=summary_dict)
summary_df






,Total Schools,Total Students,Total Budget,Avg Math Score,% Passing Math,% Passing Reading,% Passing Both
0,15,39170,24649428,78.985371,75,86,65


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [35]:
#Get total students
school_groupby= school_data_complete.groupby("school_name")
school_totals_count= school_groupby.count()
school_totals_sum= school_groupby.sum()

school_totals_sum= school_totals_sum.sort_values(by="school_name", ascending=False)



In [36]:
school_data= school_data.set_index("school_name")

In [46]:
#get school type and budget
school_data= school_data.sort_values(by="school_name", ascending=False)

#add total students

school_data= school_data.rename(columns= {"size":"Total Students","type":"Type","budget":"Budget"})
school_data["Budget Per Student"]= school_data["Budget"]/school_data["Total Students"]
school_math_total= school_totals_sum["math_score"]
school_reading_total=school_totals_sum["reading_score"]
school_data["Average Math Score"]= school_math_total/school_data["Total Students"]
school_data["Average Reading Score"]=school_reading_total/school_data["Total Students"]


#get total passing reading by school
readPass_by_school= total_passing_reading.groupby("school_name")
readPass_by_school_clean= readPass_by_school.count()["reading_score"]

#get total passing math by school
mathPass_by_school= total_passing_math.groupby("school_name")
mathPass_by_school_clean= mathPass_by_school.count()["math_score"]

#get total passing both
bothPass_by_school= total_passing_both.groupby("school_name")
bothPass_by_school_clean= bothPass_by_school.count()["Student ID"]

#add students passing per school totals to "school_data"
school_data_2= school_data.merge(readPass_by_school_clean, on="school_name", how="left")
school_data_3= school_data_2.rename(columns={"reading_score":"Total Passing Reading"})
school_data_4= school_data_3.merge(mathPass_by_school_clean, on="school_name", how="left")
school_data_5= school_data_4.rename(columns={"math_score":"Total Passing Math"})
school_data_6= school_data_5.merge(bothPass_by_school_clean, on="school_name", how="left")
school_data_7= school_data_6.rename(columns={"Student ID":"Total Passing Overall"})



# #add % passing per school to "school_data"
school_data_7["% Passing Math"]= (school_data_7["Total Passing Math"]/school_data_7["Total Students"])*100

school_data_7["% Passing Reading"]= school_data_7["Total Passing Reading"]/school_data_7["Total Students"]*100

school_data_7["% Passing Overall"]= school_data_7["Total Passing Overall"]/school_data_7["Total Students"]*100

school_overview_df= school_data_7[["Type","Budget","Total Students","Budget Per Student", "Average Math Score",
                                      "Average Reading Score","% Passing Math","% Passing Reading","% Passing Overall"]]

#mapping/formatting 
#file_df["avg_cost"] = file_df["avg_cost"].map("${:.2f}".format)

school_overview_df["Total Students"]= school_overview_df["Total Students"].map("{:,}".format)
school_overview_df["Budget"]= school_overview_df["Budget"].map("${:,}".format) 
school_overview_df["Budget Per Student"]= school_overview_df["Budget Per Student"].map("${:,.0f}".format)
school_overview_df["Average Math Score"]= school_overview_df["Average Math Score"].map("{:.2f}".format)
school_overview_df["Average Reading Score"]= school_overview_df["Average Reading Score"].map("{:.2f}".format)
school_overview_df["% Passing Math"]= school_overview_df["% Passing Math"].map("{:.2f} %".format)
school_overview_df["% Passing Reading"]= school_overview_df["% Passing Reading"].map("{:.2f} %".format)
school_overview_df["% Passing Overall"]= school_overview_df["% Passing Overall"].map("{:.2f} %".format)

school_overview_df.head()



,Type,Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Wright High School,Charter,"$1,049,400","1,800",$583,83.68,83.95,93.33 %,96.61 %,90.33 %
Wilson High School,Charter,"$1,319,574","2,283",$578,83.27,83.99,93.87 %,96.54 %,90.58 %
Thomas High School,Charter,"$1,043,130","1,635",$638,83.42,83.85,93.27 %,97.31 %,90.95 %
Shelton High School,Charter,"$1,056,600","1,761",$600,83.36,83.73,93.87 %,95.85 %,89.89 %
Rodriguez High School,District,"$2,547,363","3,999",$637,76.84,80.74,66.37 %,80.22 %,52.99 %


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [47]:
school_overview_df.sort_values(by="% Passing Overall", ascending=False).head(5)

,Type,Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356","1,858",$582,83.06,83.98,94.13 %,97.04 %,91.33 %
Thomas High School,Charter,"$1,043,130","1,635",$638,83.42,83.85,93.27 %,97.31 %,90.95 %
Griffin High School,Charter,"$917,500","1,468",$625,83.35,83.82,93.39 %,97.14 %,90.60 %
Wilson High School,Charter,"$1,319,574","2,283",$578,83.27,83.99,93.87 %,96.54 %,90.58 %
Pena High School,Charter,"$585,858",962,$609,83.84,84.04,94.59 %,95.95 %,90.54 %


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [48]:
school_overview_df.sort_values(by="% Passing Overall", ascending=True).head(5)

,Type,Budget,Total Students,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Rodriguez High School,District,"$2,547,363","3,999",$637,76.84,80.74,66.37 %,80.22 %,52.99 %
Figueroa High School,District,"$1,884,411","2,949",$639,76.71,81.16,65.99 %,80.74 %,53.20 %
Huang High School,District,"$1,910,635","2,917",$655,76.63,81.18,65.68 %,81.32 %,53.51 %
Hernandez High School,District,"$3,022,020","4,635",$652,77.29,80.93,66.75 %,80.86 %,53.53 %
Johnson High School,District,"$3,094,650","4,761",$650,77.07,80.97,66.06 %,81.22 %,53.54 %


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
student_data= student_data.set_index("school_name")

In [9]:
#create the series'
grade_nine= student_data.loc[student_data["grade"] == "9th", "math_score"] 
grade_ten= student_data.loc[student_data["grade"] == "10th", "math_score"]
grade_eleven= student_data.loc[student_data["grade"] == "11th", "math_score"]
grade_twelve= student_data.loc[student_data["grade"] == "12th", "math_score"]


In [10]:
#group the series' by school
grade_nine_schoolAve= grade_nine.groupby("school_name").mean()
grade_ten_schoolAve= grade_ten.groupby("school_name").mean()
grade_eleven_schoolAve= grade_eleven.groupby("school_name").mean()
grade_twelve_schoolAve= grade_twelve.groupby("school_name").mean()

In [11]:
#combine the series' into a dataframe

school_mathGrades_df= pd.merge(grade_nine_schoolAve, grade_ten_schoolAve, on="school_name", how="outer", suffixes=("_ave_9th","_ave_10th"))
school_mathGrades_df= pd.merge(school_mathGrades_df, grade_eleven_schoolAve, on="school_name", how= "outer")
school_mathGrades_df= pd.merge(school_mathGrades_df, grade_twelve_schoolAve, on= "school_name", how= "outer", suffixes=("_ave_11th","_ave_12th"))
school_mathGrades_df.head()



,math_score_ave_9th,math_score_ave_10th,math_score_ave_11th,math_score_ave_12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [12]:
#create the series'

grade_nine_r= student_data.loc[student_data["grade"] == "9th", "reading_score"]
grade_ten_r= student_data.loc[student_data["grade"] == "10th", "reading_score"]
grade_eleven_r= student_data.loc[student_data["grade"] == "11th", "reading_score"]
grade_twelve_r= student_data.loc[student_data["grade"] == "12th", "reading_score"]


In [13]:
#group the series' by school
grade_nine_r_schoolAve= grade_nine_r.groupby("school_name").mean()
grade_ten_r_schoolAve= grade_ten_r.groupby("school_name").mean()
grade_eleven_r_schoolAve= grade_eleven_r.groupby("school_name").mean()
grade_twelve_r_schoolAve= grade_twelve_r.groupby("school_name").mean()


In [14]:
#combine the serie' into a dataframe

school_readGrades_df= pd.merge(grade_nine_r_schoolAve, grade_ten_r_schoolAve, on="school_name", how="outer", suffixes=("_ave_9th","_ave_10th"))
school_readGrades_df= pd.merge(school_readGrades_df, grade_eleven_r_schoolAve, on="school_name", how="outer")
school_readGrades_df= pd.merge(school_readGrades_df, grade_twelve_r_schoolAve, on= "school_name", how= "outer", suffixes=("_ave_11th","_ave_12th"))
school_readGrades_df.head()

,reading_score_ave_9th,reading_score_ave_10th,reading_score_ave_11th,reading_score_ave_12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [53]:
#creating bins and bin names for school_data_7

bins= [577,597,616.5,635.75,656]
group_names=["low","medium","high","very high"]
scores_by_spending= school_data_7
scores_by_spending["Bucketed Spending per Student"]= pd.cut(school_data_7["Budget Per Student"], bins, labels= group_names)
grouped_scores_by_spending=scores_by_spending.groupby("Bucketed Spending per Student")
grouped_scores_by_spending=grouped_scores_by_spending[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                            "% Passing Overall"]].mean()
grouped_scores_by_spending



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bucketed Spending per Student,,,,,
low,83.455399,83.933814,93.460096,96.610877,90.369459
medium,83.599686,83.885211,94.230858,95.900287,90.216324
high,80.199966,82.425360,80.036217,89.536122,72.620869
very high,77.866721,81.368774,70.347325,82.995575,58.858741


## Scores by School Size

* Perform the same operations as above, based on school size.

In [16]:
#creating the bins and bin names
size_bins = [427,1564,2701,3838,4976]
size_groups= ["Very Small","Small","Medium","Large"]
school_data_size= school_data_7
school_data_size["School Size"]= pd.cut(school_data_size["Total Students"], size_bins, labels= size_groups)
school_data_size= school_data_size.groupby("School Size")
school_data_size= school_data_size[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                     "% Passing Overall"]].mean()
school_data_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Very Small,83.595708,83.930728,0.939935,0.965425,0.905700
Small,83.359224,83.898984,0.936948,0.966708,0.906182
Medium,76.814591,81.029000,0.666607,0.804516,0.536694
Large,77.063340,80.919864,0.664643,0.810597,0.536743


## Scores by School Type

* Perform the same operations as above, based on school type

In [17]:
#filter by school types 'Charter' and 'District'
schools_by_type= school_data_7.groupby("Type").mean()
schools_by_type= schools_by_type[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                 "% Passing Overall"]]
schools_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.904322
District,76.956733,80.966636,0.665485,0.807991,0.536722
